# Similarity Classification Using Sentence Pair Comparison with RoBERTa
We have generated the labelled dataset from the previous notebook create_label.ipynb.
<br>The datasets are split into training set and the validation set and put into variables train_df and eval_df respectively.

[simpletransformer](https://simpletransformers.ai/) is a wrapper to Huggingface's transformer library which helps to standardize many of the training tasks and has a nice integration with Wegihts and Bias (WandB) visualization.

The task of finding the candidate from the resume is essentially [GLUE](https://gluebenchmark.com/) task [Semantic Textual Similarity Benchmark (STS-B)]

In here I am using the **ClassificationModel** with _regression=**True**_ which is suitable for providing a single value, in this case the similarity of two sentences.

Instead of the original BERT for the pretrained model, I chose RoBERTa which uses the same engine but a different training method and more training time.
<br>It performs reasonably well even with a trimmed-down training/eval set to allow this runnable for the project submission.
<br>Comparatively, the original BERT base pretrained model (bert-base-cased) has a limited pretrained vocab and left lots of tokens marked as unknown \[UNK\]. Instead of using the BERT large I have picked the RoBERTa base (roberta-base) instead, which has a vocab of over 50k and covered most of the words used.

In [1]:
# Read back the training and eval set we have created in create_label.ipynb.
import pandas as pd
suffix = "_submit"
train_df = pd.read_excel(f"D:\\PycharmProjects\\ISMT-S117-project\\data\\train_df{suffix}.xlsx")
eval_df = pd.read_excel(f"D:\\PycharmProjects\\ISMT-S117-project\\data\\eval_df{suffix}.xlsx")

In [2]:
train_df

,Unnamed: 0,title_job,text_a,title_res,text_b,labels
0,0,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
1,4,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
2,5,c/ c++ software engineer,Dom Daniel Armenia is looking for dynamic self...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
3,6,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074
4,10,java/j2ee developer,PointSource is seeking full time J2EE Develope...,sr. java / j2ee developer,Responsibilities: \n• Involved in Requirements...,4.280726
...,...,...,...,...,...,...
3839,7611,flash/ as3 developer,NY based social and mobile games startup is ge...,developer & solution analyst,"Domain Banking \nTechnology MVC, C#.net, JQuer...",1.575380
3840,7613,unix systems administrator,Administration of Corporate Unix servers Plan...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,0.273131
3841,7614,unix systems administrator,Administration of corporate Unix Solaris Linu...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,0.273131
3842,7615,unix systems administrator,ArmenTel is seeking for candidates to fulfill ...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,0.273131


In [3]:
eval_df.shape

(1914, 6)

In [4]:
!wandb login 20427a748e4feb47468643e4770a018a2152cc0c
import wandb
id = wandb.util.generate_id()
wandb.init(project='vec2rec-roberta', id=id, resume="allow")

Successfully logged in to Weights & Biases!


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kerkermui/.netrc


W&B Run: https://app.wandb.ai/crispin/vec2rec-roberta/runs/1c109ury

In [5]:
'''
Initialize the model. Points of note:
1) class ClassificationModel is selected with roberta-base as the pretrained model.
2) It will take in evaluation data to determine / save the best model under outputs/best_model, happens every 200 steps.
3) The model is trained on GPU. If run on Kaggle GPUs, it can achieve train_batch_size of 16. My home PC can only use 6.
4) The max_seq_length determines the max length of the sentence pairs, based on previous statistics calculated 512 is enough.
5) The epoch is set to 5 which is often enough for larger datasets. It will also use the evaluation data performance to
   determine whether it can be stopped if there are no additional drops in the loss function. Given the small value of epochs
   this is also set small to avoid it from stopping prematurely.
6) The learning rate not shown here takes the default of 4e-5. However the model will also automatically adjust lr by default.
7) WandB is configured to plot the progress of the training (shown below).
'''
from simpletransformers.classification import ClassificationModel
train_args = {
    'reprocess_input_data': True,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 200,
    'max_seq_length': 512,
    'num_train_epochs': 5,
    'train_batch_size': 6,
    #'train_batch_size': 16,
    'wandb_project': 'vec2rec-roberta',
    'wandb_kwargs':{"id":id, "resume":True},
    #'no-cache':True,
    #'no_save':True,
    #'save_model_every_epoch':False,
    'save_eval_checkpoints':False,
    'save_steps':False,
    #'best_model_dir':"/kaggle/tmp/outputs/best_model"
    #'output_dir':"/kaggle/tmp/outputs/",
    'overwrite_output_dir':True,
    'use_early_stopping':True,
    'early_stopping_delta':0.001,
    'early_stopping_consider_epochs':True,
    'regression': True,
}
model = ClassificationModel('roberta', 'roberta-base', num_labels=1, args=train_args, use_cuda=True)
model.args

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

ClassificationArgs(adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', custom_layer_parameters=[], custom_parameter_groups=[], train_custom_parameters_only=False, config={}, dataloader_num_workers=6, do_lower_case=False, early_stopping_consider_epochs=True, early_stopping_delta=0.001, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=True, evaluate_during_training_silent=True, evaluate_during_training_steps=200, evaluate_during_training_verbose=False, fp16=True, fp16_opt_level='O1', gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=512, multiprocessing_chunksize=500, n_gpu=1, no_cache=False, no_save=False, num_train_epochs=5, output_dir='outputs/', overwrite_output_dir=True, process_count=6, reprocess_input_data=True, save_best_model=True, save_eval_checkpoints=Fals

In [6]:
# The Pearson and Spearman correlations are metrics used for STS-B
# https://gluebenchmark.com/tasks
from scipy.stats import pearsonr, spearmanr

def pearson_corr(preds, labels):
    return pearsonr(preds, labels)[0]

def spearman_corr(preds, labels):
    return spearmanr(preds, labels)[0]

In [7]:
import pytz
from datetime import datetime

print(datetime.now(pytz.timezone('Hongkong')))

2020-08-04 18:35:14.768148+08:00


In [8]:
%%time
%%wandb
# The model is trained and timed here with statistics shown in WandB websites displayed. The sample run took around 25 mins.
model.train_model(train_df, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


D:\ProgramData\Anaconda3\envs\ISMT-S117-project\lib\site-packages\torch\optim\lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


D:\ProgramData\Anaconda3\envs\ISMT-S117-project\lib\site-packages\torch\optim\lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0


D:\ProgramData\Anaconda3\envs\ISMT-S117-project\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)




Wall time: 25min 22s


In [9]:
# Get the final results on the eval dataset. Given this is not a discrete value classification, we can ignore the wrong_predictions
result, model_outputs, wrong_predictions = model.eval_model(eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

In [10]:
# The final loss is 0.039 with both correlations higher than 94% which is quite good given the small dataset
result

{'pearson_corr': 0.9720077410907735,
 'spearman_corr': 0.9408182334139678,
 'eval_loss': 0.03978593066065817}

In [11]:
import pytz
from datetime import datetime

print(datetime.now(pytz.timezone('Hongkong')))

2020-08-04 19:01:02.996000+08:00


In [12]:
pd.DataFrame(model_outputs).describe()

,0
count,1914.000000
mean,2.187199
std,0.828061
min,0.138184
25%,2.162598
50%,2.386719
75%,2.640625
max,4.136719


In [13]:
# Here we calculate the per row error in the eval dataset into the compare column
compare_df = pd.concat([eval_df.loc[:, "title_job":"labels"], pd.DataFrame(model_outputs, index=eval_df.index, columns=["predict"])], axis=1)
compare_df["compare"] = abs(compare_df.labels - compare_df.predict)
compare_df

,title_job,text_a,title_res,text_b,labels,predict,compare
0,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.136719,0.182355
1,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.136719,0.182355
2,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.093750,0.014346
3,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.097656,0.010440
4,c/ c++ software engineer,Dom Daniel Armenia is looking for dynamic self...,c++ developer,"Environment: CPP, Linux, JavaScript, GIT , RTC...",4.108096,4.019531,0.088565
...,...,...,...,...,...,...,...
1909,java intern,Internship opportunities are currently availab...,associate software developer,• Worked in the Feature Development of SAP Ana...,0.387010,0.370361,0.016648
1910,java intern,Internship opportunities are currently availab...,associate software developer,LPU UMS is a smart home grown web-based ERP (E...,0.387010,0.375732,0.011277
1911,java intern,Internship opportunities are currently availab...,associate software developer,• Businesses to Riders Group Project– Django +...,0.387010,0.383057,0.003953
1912,graphic designer / customer service operator,Time to Print salon of operative printing is l...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,1.577018,1.166992,0.410025


In [14]:
# The eval loss above is essentially the mean_squared_error, while we can also calculate the mean_absolute error
# In this case we can see that for all of the eval set on average we have an error of 0.064 on a 5 point scale, which is not bad.
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(mean_squared_error(compare_df.labels, compare_df.predict))
print(mean_absolute_error(compare_df.labels, compare_df.predict))

0.03961561556822494
0.0647740035493982


In [15]:
compare_df.compare.describe()

count    1914.000000
mean        0.064774
std         0.188251
min         0.000002
25%         0.005830
50%         0.013425
75%         0.034553
max         2.468148
Name: compare, dtype: float64

In [21]:
# Only 40 samples have a score error > 0.5
pd.cut(compare_df.compare, bins=[-1,0.5,1,2,3,4,6], right=True).value_counts()

(-1.0, 0.5]    1864
(0.5, 1.0]       32
(1.0, 2.0]       15
(2.0, 3.0]        3
(4.0, 6.0]        0
(3.0, 4.0]        0
Name: compare, dtype: int64

In [18]:
compare_df[compare_df.compare < 0.5]

,title_job,text_a,title_res,text_b,labels,predict,compare
0,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.136719,0.182355
1,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,"c,c++ developer","Languages Known: C, C++, Data Structures, \nJa...",4.319074,4.136719,0.182355
2,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.093750,0.014346
3,c/ c++ software engineer,LTX Credence Armenia LLC is looking for Softwa...,c++ developer,"Atlanta, USA Aug 2011 - Sep 2012 \nProject Nam...",4.108096,4.097656,0.010440
4,c/ c++ software engineer,Dom Daniel Armenia is looking for dynamic self...,c++ developer,"Environment: CPP, Linux, JavaScript, GIT , RTC...",4.108096,4.019531,0.088565
...,...,...,...,...,...,...,...
1909,java intern,Internship opportunities are currently availab...,associate software developer,• Worked in the Feature Development of SAP Ana...,0.387010,0.370361,0.016648
1910,java intern,Internship opportunities are currently availab...,associate software developer,LPU UMS is a smart home grown web-based ERP (E...,0.387010,0.375732,0.011277
1911,java intern,Internship opportunities are currently availab...,associate software developer,• Businesses to Riders Group Project– Django +...,0.387010,0.383057,0.003953
1912,graphic designer / customer service operator,Time to Print salon of operative printing is l...,senior developer/ business analyst,12-Sep-2012 Present Senior Developer/ Business...,1.577018,1.166992,0.410025


In [19]:
pd.options.display.max_colwidth = None
print(compare_df[compare_df.compare >= 1].shape)
compare_df[compare_df.compare >= 1]

(18, 7)


,title_job,text_a,title_res,text_b,labels,predict,compare
7,senior software developer/ technical leader,As Senior Software Developer Technical Leader the incumbent has to lead and work with developers and participate in the design and development projects he has to work with Architecture team in the organization understand product roadmaps and assume key role in the development of these products BS MS in Computer Science Information Services or related field Minimum of 10 year extensive development experience Experience as Technical Leader on in Linux OS with in depth knowledge of software development processes Deep technical and professional skills readiness to learn new technologies Extensive programming experience with standard management and instrumentation API Ability to work independently as well as with other members in the group preferably in leadership role Ability to write technical and functional specifications and present them to the other team members and occasionally to the management thorough understanding of software development processes and methodologies and knowledge of complete product development life cycle Strong inter personal skills ability to work both independently and as part of global team and thrive on technical challenges Excellent communication and writing skills and experience in writing product specifications and technical documentation Good communication skills in English language Enthusiastic and self motivated Good team player,"technical lead, developer","for 2013 aimed at Life and Pensions products, intended to bring about changes in projection rates, to be effective from January, 2014. \n \nRole Technical lead, Developer \nResponsibilities \n- Writing Technical Specification document using Class Diagrams and Sequence Diagrams \n \n- Technical lead to guide developers through the development, unit and integration testing, UAT phases of the project \n \n- Review and upgrade of code developed by team members \n \n- Coordinating with onsite coordinator in project testing and project implementation phases \n \n- Ensuring quality of deliverables to be compliant with AEGON standard and Integrated Quality Management System standard of Tata Consultancy Services, through Internal and External Quality Assurance checks \n \nSolution Environment Java/J2EE (Spring framework v2.5.6), SQL, Rational Application Developer, Rational Clearcase, TOAD",4.100830,2.619141,1.481689
8,senior software developer/ technical leader,As Senior Software Developer Technical Leader the incumbent has to lead and work with developers and participate in the design and development projects he has to work with Architecture team in the organization understand product roadmaps and assume key role in the development of these products BS MS in Computer Science Information Services or related field Minimum of 10 year extensive development experience Experience as Technical Leader on in Linux OS with in depth knowledge of software development processes Deep technical and professional skills readiness to learn new technologies Extensive programming experience with standard management and instrumentation API Ability to work independently as well as with other members in the group preferably in leadership role Ability to write technical and functional specifications and present them to the other team members and occasionally to the management thorough understanding of software development processes and methodologies and knowledge of complete product development life cycle Strong inter personal skills ability to work both independently and as part of global team and thrive on technical challenges Excellent communication and writing skills and experience in writing product specifications and technical documentation Good communication skills in English language Enthusiastic and self motivated Good team player,"technical lead, developer","Kolkata, India \nCustomer Grameen Phone, Bangladesh \nPeriod December 2014 - March 2016 \nDescription \n